In [275]:
#Update your token
STUDENT_TOKEN = 'ASWIN-SUBRAMANIANMAHESWARAN'

Server communication game and game_loop.

In [276]:
## ignore this code, just used for submission

import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
    def __init__(self, state, status, player):
        self.state = state
        self.status = status
        self.player = player

    def is_waiting(self):
        return self.status == 'waiting'

    def is_end(self):
        return self.status == 'complete'

    def get_board(self):
        print(self.state)
        return json.loads(self.state)

    def actions(self):
        return []

    def print_game(self):
        print(self.state)

def new_game(game_type, multi_player = False):
    for _ in range(10):
        r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
        if r.status_code == 200:
            return r.json()['game-id']
        print(r.content)

def join_game(game_type, game_id):
    for _ in range(10):
        r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
        if r.status_code == 200:
          return r.json()['player']
        print(r.content)

def game_state(game_type, game_id, GameClass, debug_mode=False):
    for _ in range(10):
        r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
        if r.status_code == 200:
           return GameClass(r.json()['state'], r.json()['status'], r.json()['player'], debug_mode)
        print(r.content)

def update_game(game_type, game_id, player, move):
    for _ in range(10):
        r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
        if r.status_code == 200:
          return r.content
        print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None, debug_mode=False):
    while id == None:
        print('\033[93mCreating new game...\033[0m')
        id = new_game(game_type, multi_player)

    print('\033[93mJoining game with id: \033[0m%s' % id)
    player = join_game(game_type, id)

    print('\033[93mPlaying as %s\033[0m' % player)

    game = game_state(game_type, id, GameClass, debug_mode)
    print('\033[93mWaiting for the other player to join...\033[0m')
    while game.is_waiting():
        time.sleep(10)
        game = game_state(game_type, id, GameClass)

    while True:
        game = game_state(game_type, id, GameClass, debug_mode)
        game.print_game()
        if game.is_end():
            if game.player == '-':
                print('\033[94mDRAW\033[0m')
            else:
                print('\033[92mWINNER\033[0m' if game.player == player else '\033[91mLOSER\033[0m')
            return
        if game.player == player:
            if not debug_mode: print('Making next move...')
            move = solver(game)
            if debug_mode: print(f"LAST MOVE: {move}")
            update_result = update_game(game_type, id, player, json.dumps(move))
            # print(f"UPDATE RESULT: {update_result}")
        else:
            time.sleep(2)

Random solver

In [277]:
def random_solver(game):
    actions = game.actions()
    action = random.choice(actions)
    return action

Game specific code

In [ ]:
class NineMensMorris(Game):
    
    MILLS = [
        {0, 1, 2}, {3, 4, 5}, {6, 7, 8}, {9, 10, 11},
        {12, 13, 14}, {15, 16, 17}, {18, 19, 20},
        {21, 22, 23}, {0, 9, 21}, {3, 10, 18},
        {6, 11, 15}, {1, 4, 7}, {16, 19, 22},
        {8, 12, 17}, {5, 13, 20}, {2, 14, 23}
    ]
    
    ADJACENT_POINTS = {
        0: [1, 9],
        1: [0, 2, 4],
        2: [1, 14],
        3: [4, 10],
        4: [1, 3, 5, 7],
        5: [4, 13],
        6: [7, 11],
        7: [4, 6, 8],
        8: [7, 12],
        9: [0, 10, 21],
        10: [3, 9, 11, 18],
        11: [6, 10, 15],
        12: [8, 13, 17],
        13: [5, 12, 14, 20],
        14: [2, 13, 23],
        15: [11, 16],
        16: [15, 17, 19],
        17: [12, 16],
        18: [10, 19],
        19: [16, 18, 20, 22],
        20: [13, 19],
        21: [9, 22],
        22: [19, 21, 23],
        23: [14, 22]
    }

    def __init__(self, state, status, player, debug_mode=False):
        super().__init__(state, status, player)
        self.debug_mode = debug_mode

        # Convert state to a dictionary if it's a JSON string
        if isinstance(state, str):
            state = json.loads(state)

        self.board = state.get('board', ['.'] * 24)
        self.actions_list = state.get('actions', [])
        self.phase = state.get('phase', 'placing')
        self.update_pieces()

    def other_player(self):
        return 'O' if self.player == 'X' else 'X'

    def is_mill(self, board, index):
        player = board[index]
        for mill in self.MILLS:
            if index in mill and all(board[i] == player for i in mill):
                return True
        return False

    def actions(self):
        return self.actions_list

    def update_pieces(self):
        self.pieces_on_board = {'X': self.board.count('X'), 'O': self.board.count('O')}
        self.pieces_to_place = {
            'X': max(0, 9 - self.pieces_on_board['X']),
            'O': max(0, 9 - self.pieces_on_board['O'])
        }

    def is_terminal(self):
        if self.phase == 'placing':
            return False
        opponent = self.other_player()
        if self.pieces_on_board[opponent] < 3:
            return True
        if not self.has_moves(opponent):
            return True
        return False

    def has_moves(self, player):
        if self.phase == 'placing':
            return any(spot == '.' for spot in self.board)
        elif self.phase == 'moving':
            player_pieces = [i for i, spot in enumerate(self.board) if spot == player]
            for pos_from in player_pieces:
                if any(self.board[pos_to] == '.' for pos_to in self.ADJACENT_POINTS.get(pos_from, [])):
                    return True
        elif self.phase == 'flying':
            player_pieces = [i for i, spot in enumerate(self.board) if spot == player]
            return bool(player_pieces and any(spot == '.' for spot in self.board))
        return False

    def print_game(self):
        print("Board:")
        self.print_board()
        if self.debug_mode:
            print(f"ACTIONS: {self.actions_list}")
        if not self.debug_mode:
            print(f"Player: {self.player}")
        if self.debug_mode:
            print(f"PIECES ON BOARD: {self.pieces_on_board}")
            print(f"PIECES TO PLACE: {self.pieces_to_place}")
        if not self.debug_mode:
            print(f"Phase: {self.phase}")
        print()

    def print_board(self):
        board_layout = [
            [0, None, None, 1, None, None, 2],
            [None, 8, None, 9, None, 10, None],
            [None, None, 16, 17, 18, None, None],
            [7, 15, 23, None, 19, 11, 3],
            [None, None, 22, 21, 20, None, None],
            [None, 14, None, 13, None, 12, None],
            [6, None, None, 5, None, None, 4]
        ]

        for row in board_layout:
            line = []
            for index in row:
                if index is None:
                    line.append(" ")
                else:
                    if self.board[index] == self.player:
                        line.append(f"\033[92m{self.board[index]}\033[0m")
                    elif self.board[index] in ['X', 'O']:
                        line.append(f"\033[91m{self.board[index]}\033[0m")
                    else:
                        line.append(".")
            print(" ".join(line))


In [279]:
# Define the Solver for the game

MAX_DEPTH = 5

def alpha_beta_solver(game):
    """
    Alpha-beta pruning solver for Nine Men's Morris with optimized depth and heuristics.
    """
    def max_value(state, alpha, beta, depth):
        if depth >= MAX_DEPTH or state.is_terminal():
            return evaluate_state(state), None
        value, best_action = float('-inf'), None
        for action in state.actions():
            new_state = simulate_action(state, action)
            min_val, _ = min_value(new_state, alpha, beta, depth + 1)
            if min_val > value:
                value, best_action = min_val, action
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return value, best_action

    def min_value(state, alpha, beta, depth):
        if depth >= MAX_DEPTH or state.is_terminal():
            return evaluate_state(state), None
        value, best_action = float('inf'), None
        for action in state.actions():
            new_state = simulate_action(state, action)
            max_val, _ = max_value(new_state, alpha, beta, depth + 1)
            if max_val < value:
                value, best_action = max_val, action
            beta = min(beta, value)
            if beta <= alpha:
                break
        return value, best_action

    _, best_action = max_value(game, float('-inf'), float('inf'), 0)
    return best_action if best_action is not None else random.choice(game.actions())

def evaluate_state(state):
    """
    Heuristic evaluation function for Nine Men's Morris.
    EVAL(s) =   W1 * (Agent's Pieces - Opponent's Pieces)
              + W2 * (Agent's Moves - Opponent's Moves)
              + W3 * (Agent's Mills - Opponent's Mills)
    """
    W1 = 10  # No of pieces
    W2 = 5   # No of possible moves
    W3 = 15  # No of mills

    player = state.player
    opponent = state.other_player()

    # Number of pieces for player and opponent
    player_pieces = state.pieces_on_board[player]
    opponent_pieces = state.pieces_on_board[opponent]
    
    # Number of possible moves for player and opponent
    player_moves = len(get_all_possible_moves(state, player))
    opponent_moves = len(get_all_possible_moves(state, opponent))
    
    # Number of mills for player and opponent
    player_mills = count_mills(state, player)
    opponent_mills = count_mills(state, opponent)

    # Evaluation formula
    eval_value = (W1 * (player_pieces - opponent_pieces)
                  + W2 * (player_moves - opponent_moves)
                  + W3 * (player_mills - opponent_mills))
    
    return eval_value

def get_all_possible_moves(state, player):
    """
    Returns a list of all possible moves for a given player.
    This function is useful for evaluating mobility.
    """
    possible_moves = []
    for action in state.actions():
        # Assuming actions are in the form of [move_from, move_to, remove]
        # Check if the player can make the move
        if (player == state.player and (action[0] == -1 or state.board[action[0]] == player)):
            possible_moves.append(action)
    return possible_moves

def count_mills(state, player):
    """
    Count the number of mills formed by a given player.
    """
    count = 0
    for mill in state.MILLS:
        if all(state.board[i] == player for i in mill):
            count += 1
    return count


def simulate_action(state, action):
    new_state = deepcopy(state)
    move_from, move_to, remove = action

    player = state.player
    opponent = state.other_player()

    if move_from == -1:  # Placing phase
        new_state.board[move_to] = player
        new_state.pieces_to_place[player] -= 1
        new_state.pieces_on_board[player] += 1
        if remove != -1:
            new_state.board[remove] = '.'
            new_state.pieces_on_board[opponent] -= 1
    else:  # Moving/Flying phase
        new_state.board[move_from] = '.'
        new_state.board[move_to] = player
        if remove != -1:
            new_state.board[remove] = '.'
            new_state.pieces_on_board[opponent] -= 1

    new_state.update_pieces()
    if state.phase == 'placing' and new_state.pieces_to_place['X'] == 0 and new_state.pieces_to_place['O'] == 0:
        new_state.phase = 'moving'
    elif state.phase == 'moving' and new_state.pieces_on_board[player] <= 3:
        new_state.phase = 'flying'

    return new_state

In [280]:
# Run the game loop

game_loop(alpha_beta_solver, NineMensMorris, 'ninemensmorris', multi_player=False, id=None, debug_mode=True)

Creating new game...
Joining game with id: 24171
Playing as X
Waiting for the other player to join...
Board:
.     .     .
  .   .   .  
    . . .    
. . .   . . .
    . . .    
  .   .   .  
.     .     .
ACTIONS: [[-1, 0, -1], [-1, 1, -1], [-1, 2, -1], [-1, 3, -1], [-1, 4, -1], [-1, 5, -1], [-1, 6, -1], [-1, 7, -1], [-1, 8, -1], [-1, 9, -1], [-1, 10, -1], [-1, 11, -1], [-1, 12, -1], [-1, 13, -1], [-1, 14, -1], [-1, 15, -1], [-1, 16, -1], [-1, 17, -1], [-1, 18, -1], [-1, 19, -1], [-1, 20, -1], [-1, 21, -1], [-1, 22, -1], [-1, 23, -1]]
PIECES ON BOARD: {'X': 0, 'O': 0}
PIECES TO PLACE: {'X': 9, 'O': 9}

LAST MOVE: [-1, 0, -1]
Board:
X     .     .
  .   .   .  
    . . .    
. . .   . . .
    . . .    
  .   O   .  
.     .     .
ACTIONS: [[-1, 1, -1], [-1, 2, -1], [-1, 3, -1], [-1, 4, -1], [-1, 5, -1], [-1, 6, -1], [-1, 7, -1], [-1, 8, -1], [-1, 9, -1], [-1, 10, -1], [-1, 11, -1], [-1, 12, -1], [-1, 14, -1], [-1, 15, -1], [-1, 16, -1], [-1, 17, -1], [-1, 18, -1], [-1, 19, -1], [-1, 20